In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
pd.set_option('display.max_columns', 10)  # or 1000
pd.set_option('display.max_rows', 10)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
# Saving chart with white background
plt.rcParams['savefig.facecolor'] = 'white'

In [5]:
analyst_data = pd.read_excel('input/LinkedIn-Job-Data-Analyst-Sep-13-2022_1923.xlsx')
engineer_data = pd.read_excel('input/LinkedIn-Job-Data-Engineer-Sep-13-2022_1808.xlsx')
scientist_data = pd.read_excel('input/LinkedIn-Job-Data-Scientist-Sep-13-2022_1728.xlsx')

# 1. Summary Data

In [6]:
analyst_data.head()

Company        Date                    Title  \
0                                Hismile  2022-09-13      Junior Data Analyst   
1  Resonance Technology Services Pty LTD  2022-09-13             Data Analyst   
2                                Datacom  2022-08-18  Graduate - Data Analyst   
3                                   Sika  2022-08-23      Junior Data Analyst   
4                    Omnicom Media Group  2022-09-09      Junior Data Analyst   

                             Location  \
0   Gold Coast, Queensland, Australia   
1  Sydney, New South Wales, Australia   
2  Sydney, New South Wales, Australia   
3  Sydney, New South Wales, Australia   
4  Sydney, New South Wales, Australia   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [7]:
analyst_data.describe()

,Company,Date,Title,Location,Description,Type,Link
count,491,491,491,491,491,491,491
unique,264,57,231,71,358,283,491
top,Agoda,2022-09-12,Data Analyst,"Sydney, New South Wales, Australia",None,"Seniority level\nNot Applicable\nEmployment type\nFull-time\nJob function\nInformation Technology\nIndustries\nFinancial Services, Business Consulting and Services, and Accounting",https://au.linkedin.com/jobs/view/junior-data-analyst-at-hismile-3260036794?refId=XbXfkkck%2Fb6mw5O5dbWijw%3D%3D&trackingId=Hv8%2F6dNqK7eKim71t9SLjA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card
freq,33,47,105,169,59,19,1


# 2. Data Cleansing

In [8]:
def add_job_type(df, job_title):
    idx = analyst_data.columns.get_loc('Title')
    try:
        df.insert(loc=idx+1, column='Job Type', value=job_title)
    except:
        pass
    return df

def drop_metrics(df):
    df = df.copy()
    df = df.drop(['Date', 'Link'], axis='columns')
    return df
    
def drop_description_null(df):
    df = df.drop(df[df.Description=='None'].index).reset_index(drop=True)
    return df

def drop_duplicate_posts(df):
    keep_idx = df[['Company', 'Title', 'Description']].drop_duplicates().index
    df = df.iloc[keep_idx].reset_index(drop=True)
    return df

def filter_title(df, job_title):
    df = df[df['Title'].str.contains(job_title, regex=True)].reset_index(drop=True)
    return df

In [9]:
# Data Filtering
def filter_data(df, job_title):
    df = add_job_type(df, job_title)
    df = drop_metrics(df)
    df = drop_description_null(df)
    df = drop_duplicate_posts(df)
    df = filter_title(df, job_title)
    return df

In [10]:
def split_location(df):
    df_temp = df["Location"].str.split(',', expand=True)
    df_temp.columns=['City', 'State', 'Country']
    df = pd.concat([df.loc[:, :"Location"], df_temp, df.loc[:, "Description":]], axis='columns')
    df = df.drop(columns='Location')
    
    # Remove white space character
    df['State'] = df['State'].str.strip()
    df['Country'] = df['Country'].str.strip()
    return df

def split_type(df):
    df_type = df.Type.str.replace("\n", ";").str.split(";", 7 , expand=True)
    
    # Only keep rows that not have null value in type metrics
    df = df[df_type[7].notnull()].reset_index(drop=True)
    df_type = df_type[df_type[7].notna()].reset_index(drop=True)

    columns_name = df_type.iloc[0][[0,2,4,6]].values
    df_type = df_type.drop(df_type.columns[[0,2,4,6]], axis='columns')
    df_type.columns = columns_name
    
    # Concat type df to the end clean data
    df = pd.concat([df, df_type], axis='columns')
    df = df.drop('Type', axis='columns')
    return df

In [11]:
# Check if City, State, and Country value are in the correct place
def update_location(row):
    if (row['Country'] == None):
        if(row['City'] == 'Australia'):
            row['Country'] = row['City']
            row['City'] = ''
        else:
            row['Country'] = row['State']
            row['State'] = row['City']
            row['City'] = ''
    
    return row

def update_all_locations(df):
    df = df.apply(lambda x: update_location(x), axis=1)
    return df

In [12]:
# Spliting string in columns to multiple ones
def split_data(df):
    df = split_location(df)
    df = split_type(df)
    df = update_all_locations(df)
    return df

In [13]:
## Text Cleaning for NLP
import re
import nltk
nltk.download('wordnet', quiet=True)

# Move desc to the last columns
def move_desc_last(df):
    df = df[[c for c in df if c not in ['Description']]
       + ['Description']]
    return df

def remove_newline_desc(df):
    df.Description = df.Description.str.replace("\n", ' ').to_frame()
    return df


# Normalization
def normalize_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    return df


# Stop words
import nltk.corpus
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')

def remove_stopword(df):
    df['Description_clean'] = df['Description_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    return df

import nltk 
nltk.download('punkt')

from nltk.tokenize import sent_tokenize, word_tokenize
def tokenize_text(df):
    df['Description_tokens'] = df['Description_clean'].apply(lambda x: word_tokenize(x))
    return df

# Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

def tokens_lemmatizer(df):
    df['Description_tokens_lemma'] = df['Description_tokens'].apply(lambda x: word_lemmatizer(x))
    return df

def get_unique_tokens_lemma(df):
    df['Description_tokens_lemma_unique'] = df['Description_tokens_lemma'].apply(lambda x: sorted(set(x)))
    return df



True

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Austin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Austin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Austin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
# List of common skills and tools in Data Science
tech_list = ['sql', 'python', 'r', 'sas', 'java', 'scala', 'c', 'c++', 'javascript', 'swift', 'matlab', 'go', 'perl', 'php', 'ruby', 'html', 'excel', 'tableau', 'powerbi', 'aws', 'azure', 'jira', 'git', 'jenkins', 'matplotlib', 'pandas', 'spark', 'plotly', 'tensorflow', 'pytorch', 'seaborn', 'tidyverse', 'snowflake', 'nltk']

# Get list of skills that the job require
def get_skills_tokens(df):
    df['Skills'] = df['Description_tokens_lemma_unique'].apply(lambda x: [skill for skill in x if skill in tech_list])
    return df

# One-hot encoding column of skills list
def skill_list_oh_encode(df):
    df = df.join(df['Skills'].str.join('|').str.get_dummies())
    return df

In [15]:
len(tech_list)

34

In [16]:
# Full clean description nlp
def clean_desc(df):
    df = move_desc_last(df)
    df = remove_newline_desc(df)
    df = normalize_text(df, 'Description', 'Description_clean')
    df = remove_stopword(df)
    df = tokenize_text(df)
    df = tokens_lemmatizer(df)
    df = get_unique_tokens_lemma(df)
    df = df.drop(columns=['Description', 'Description_clean', 'Description_tokens'])  # Drop unnecessary description columns
    return df

In [17]:
def get_skills(df):
    df = get_skills_tokens(df)
    return df

In [18]:
def clean_attributes(df):
    df = df.drop(columns=['Description_tokens_lemma', 'Description_tokens_lemma_unique'])
    return df

In [19]:
def full_data_cleansing(df, job_title):
    df = filter_data(df, job_title)
    df = split_data(df)
    df = clean_desc(df)
    df = get_skills(df)
    df = clean_attributes(df)
    return df

In [20]:
analyst_data_clean = full_data_cleansing(analyst_data, job_title='Data Analyst')
engineer_data_clean = full_data_cleansing(engineer_data, job_title='Data Engineer')
scientist_data_clean = full_data_cleansing(scientist_data, job_title='Data Scientist')

In [21]:
all_data_clean = pd.concat([analyst_data_clean, engineer_data_clean, scientist_data_clean])\
                    .reset_index(drop=True)

In [22]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
all_data_clean.select_dtypes(include=numerics).fillna(0, inplace=True)

In [23]:
all_data_clean

,Company,Title,Job Type,City,State,...,Seniority level,Employment type,Job function,Industries,Skills
0,Datacom,Graduate - Data Analyst,Data Analyst,Sydney,New South Wales,...,Entry level,Full-time,Information Technology,IT Services and IT Consulting,[excel]
1,Sika,Junior Data Analyst,Data Analyst,Sydney,New South Wales,...,Not Applicable,Full-time,Information Technology,Chemical Manufacturing,[excel]
2,Omnicom Media Group,Junior Data Analyst,Data Analyst,Sydney,New South Wales,...,Entry level,Full-time,Analyst,Online Audio and Video Media,"[excel, python, r, sql]"
3,Afterpay,"Data Analyst, Marketing",Data Analyst,Sydney,New South Wales,...,Associate,Full-time,General Business,IT Services and IT Consulting,"[go, powerbi, python, sql, tableau]"
4,Lockton Australia,Junior Data Analyst,Data Analyst,Sydney,New South Wales,...,Entry level,Full-time,"Analyst, Information Technology, and Science",Insurance and Financial Services,"[azure, git, python, r, sql]"
...,...,...,...,...,...,...,...,...,...,...,...
777,Net2Source Inc.,Sr. Data Scientist,Data Scientist,Melbourne,Victoria,...,Associate,Other,Engineering and Information Technology,Staffing and Recruiting,"[aws, python, spark, tensorflow]"
778,Antler,"Founder / Co-Founder (Software Engineer, Full Stack Developer, Data Scientist)",Data Scientist,Sydney,New South Wales,...,Mid-Senior level,Full-time,Engineering and Product Management,IT Services and IT Consulting and Venture Capital and Private Equity Principals,[c]
779,Deloitte,Senior Manager or Director/ Data Scientist | Multiple Locations,Data Scientist,Sydney,New South Wales,...,Director,Full-time,Other,Legal Services,"[python, r, sql, tableau]"
780,Agoda,"Senior Data Scientist/ Senior Machine Learning Engineer (Bangkok based, relocation provided)",Data Scientist,Melbourne,Victoria,...,Mid-Senior level,Full-time,Engineering and Information Technology,"Technology, Information and Internet","[python, scala, sql]"


In [24]:
all_data_clean.columns

Index(['Company', 'Title', 'Job Type', 'City', 'State', 'Country',
       'Seniority level', 'Employment type', 'Job function', 'Industries',
       'Skills'],
      dtype='object')

In [28]:
all_data_clean.to_excel("output/data/Data_Jobs_in_Australia_cleaned.xlsx", sheet_name="Data Jobs in Australia", index=False)